WORKFLOW:
parto dal dataframe --> discretizzo valore target —> vector assembler con tutti gli attributi tranne quello targer —> standardizzo valori nel vettore creato con minmaxscaler —> preparo i dati per splittarli —> splitto in train-test —> applico random forest —> evaluation


Sito utile: [link text](https://towardsdatascience.com/uci-heart-disease-classification-with-pyspark-eadc8e99663f)

in particolare: [link text](https://medium.com/analytics-vidhya/building-a-classification-model-using-pyspark-in-databricks-909646147b57)

In [2]:
!pip install pyspark==3.1.2 py4j==0.10.9 

Processing c:\users\giamboseller\appdata\local\pip\cache\wheels\a5\0a\c1\9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77\pyspark-3.1.2-py2.py3-none-any.whl
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.0.1
    Uninstalling pyspark-3.0.1:
      Successfully uninstalled pyspark-3.0.1


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .master("local[4]") \
    .config("spark.driver.maxResultSize", "8g") \
    .getOrCreate()

In [2]:
df = spark.read.parquet('../../data/cleanedDataset_parquet')

In [3]:
df.printSchema()

root
 |-- id_track: string (nullable = true)
 |-- popularity_track: integer (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- release_date: date (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- time_signature: integer (nullable = true)
 |-- sum_artist_followers: long (nullable = true)
 |-- sum_artist_popularity: long (nullable = true)
 |-- avg_artist_followers: double (nullable = true)
 |-- avg_artist_popularity: double (nullable = true)
 |-- age: double (nullable = true)



In [4]:
from pyspark.ml.feature import Bucketizer
bucketBorders=[0,10,20,30,40,50,60,70,80,90,100]

In [5]:
bucketer=Bucketizer().setSplits(bucketBorders).setInputCol("popularity_track").setOutputCol("buckets")
df_buck = bucketer.transform(df)

In [6]:
df_buck.printSchema() # perchè df_buck.show(5) mi dà errore?

root
 |-- id_track: string (nullable = true)
 |-- popularity_track: integer (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- release_date: date (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- time_signature: integer (nullable = true)
 |-- sum_artist_followers: long (nullable = true)
 |-- sum_artist_popularity: long (nullable = true)
 |-- avg_artist_followers: double (nullable = true)
 |-- avg_artist_popularity: double (nullable = true)
 |-- age: double (nullable = true)
 |-- bucke

In [7]:
df_buck.show()

+--------------------+----------------+-----------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+---------------------+--------------------+---------------------+------------------+-------+
|            id_track|popularity_track|duration_ms|              genres|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|sum_artist_followers|sum_artist_popularity|avg_artist_followers|avg_artist_popularity|               age|buckets|
+--------------------+----------------+-----------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+---------------------+--------------------+---------------------+------------------+-------+
|00AeAaSNbe92PRrst...|               3|    

Target: *popularity_track* che ha un range (0-100)

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler( 
inputCols=[
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'avg_artist_followers',
 'avg_artist_popularity',
 'sum_artist_followers',
 'sum_artist_popularity'], 
outputCol="features")
output=assembler.transform(df_buck)

In [10]:
output.printSchema() # qui perchè se nella cella sopra passo df_cla al posto che df_buck mi fa fare .show e invece così no?

root
 |-- id_track: string (nullable = true)
 |-- popularity_track: integer (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- release_date: date (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: integer (nullable = true)
 |-- speechiness: float (nullable = true)
 |-- acousticness: float (nullable = true)
 |-- instrumentalness: float (nullable = true)
 |-- liveness: float (nullable = true)
 |-- valence: float (nullable = true)
 |-- tempo: float (nullable = true)
 |-- time_signature: integer (nullable = true)
 |-- sum_artist_followers: long (nullable = true)
 |-- sum_artist_popularity: long (nullable = true)
 |-- avg_artist_followers: double (nullable = true)
 |-- avg_artist_popularity: double (nullable = true)
 |-- age: double (nullable = true)
 |-- bucke

In [11]:
from pyspark.ml.feature import MinMaxScaler
scaler = MinMaxScaler(inputCol = "features",outputCol= "scaledFeatures")
scalerData = scaler.fit(output)
out = scalerData.transform(output)

In [12]:
final_data = out.select("id_track", "scaledFeatures", "buckets")

In [13]:
train, test = final_data.randomSplit([0.7, 0.3], seed = 10)

In [14]:
train.printSchema() # ma se faccio .show non va

root
 |-- id_track: string (nullable = true)
 |-- scaledFeatures: vector (nullable = true)
 |-- buckets: double (nullable = true)



In [15]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol="scaledFeatures",labelCol="buckets",maxDepth = 10)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)

In [16]:
predictions.show(10)

+--------------------+--------------------+-------+--------------------+--------------------+----------+
|            id_track|      scaledFeatures|buckets|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-------+--------------------+--------------------+----------+
|000TiSS4vK5su0Mko...|[0.02775177940979...|    4.0|[0.88187990551266...|[0.04409399527563...|       4.0|
|000mGrJNc2GAgQdME...|[0.08815007242953...|    0.0|[7.28871282486940...|[0.36443564124347...|       0.0|
|001e2JrYMwnTeRnxf...|[0.06424779195831...|    1.0|[11.2105833023136...|[0.56052916511568...|       0.0|
|001gx41rQo0bKh063...|[0.06740735018264...|    2.0|[2.26522938720071...|[0.11326146936003...|       2.0|
|0022sbR4gAWFhpMaY...|[0.03795332540388...|    2.0|[0.94804683574769...|[0.04740234178738...|       3.0|
|002x5YvGv3c0OW9HO...|[0.03020555462796...|    1.0|[3.63086407796632...|[0.18154320389831...|       2.0|
|003ShLCyC9ACAsdUW...|[0.04009150080617...|    1.0|[3.1

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="buckets", predictionCol="prediction",metricName="accuracy")
evaluator.evaluate(predictions)

0.383524202429658